# Final Project: Admission Prediction from NHAMCS
## Data exploration notebook
### DS5559: Big Data Analysis
### Thomas Hartka, Alicia Doan, Michael Langmayr
Created: 6/21/20  
  
In this notebook we determine which years contain data for certain variable and write this to a CSV.

## Configure

In [8]:
# set data directory
data_dir = "../data"

In [9]:
# import python libraries
import os
import pandas as pd
import numpy as np
from functools import reduce

In [10]:
# set up pyspark
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

## Read in data

In [11]:
%%time
NHAMCS = spark.read.parquet(data_dir + "/NHAMCS.2007-2017")

CPU times: user 1.6 ms, sys: 578 µs, total: 2.18 ms
Wall time: 651 ms


In [12]:
NHAMCS.count()

305897

In [13]:
# number of columns
len(NHAMCS.columns)

1219

## Look for missing data

In [8]:
%%time
# create dataframe with year and counts
years_null = pd.DataFrame(NHAMCS.groupBy("YEAR").agg(count('YEAR').alias('N')).collect(), columns=["YEAR","N"])

# find col
for col in NHAMCS.columns:
    if col !='YEAR':
        #print(col)
        n = NHAMCS.select('YEAR',col).subtract(NHAMCS.select('YEAR',col).dropna()).groupBy("YEAR").agg(count('YEAR')).collect()
        #print(n)
        col_nulls = pd.DataFrame(n,columns=["YEAR",col])
        #print(col_nulls)
        years_null = years_null.merge(col_nulls, how='left', on="YEAR")

CPU times: user 1min 37s, sys: 1.91 s, total: 1min 39s
Wall time: 51min 31s


In [10]:
# change ALL NULL flag from 1 to 0, and NOT ALL NULL from null to 1
years_data = years_null.replace(1.0,int(0)).fillna(int(1)).astype(int) \
                        .sort_values('YEAR').reset_index(drop=True)

In [11]:
years_data

,YEAR,N,VMONTH,VYEAR,VDAYR,AGE,ARRTIME,WAITTIME,LOV,RESIDNCE,...,EXCHSUM2E,BLANK7,BLANK8,EWHONOTE,EWHOPRACE,EWHOOTHE,EWHOPRACER,EXCHSUM4E,EWHOUNKE,EXCHSUME
0,2007,35490,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2008,34134,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2009,34942,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2010,34936,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2011,31084,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,2012,29453,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6,2013,24777,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,2014,23844,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,2015,21061,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,2016,19467,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [7]:
NHAMCS.filter(NHAMCS.YEAR>2014).count()

57237

## Write out varaibles table

In [14]:
years_data.to_csv("../results/NHAMCS_vars_by_year.csv")